**Table of contents**<a id='toc0_'></a>    
- [Forward Kinematics Using Homogeneous Transformation Matrix](#toc1_)    
    - [Starting pybullet](#toc1_1_1_)    
    - [Initial Setup for pybullet](#toc1_1_2_)    
    - [Generating the Robot Arm](#toc1_1_3_)    
    - [Definition of Homogeneous Transformation Matrix in a 2D Plane](#toc1_1_4_)    
    - [Executing Forward Kinematics Using Homogeneous Transformation Matrix](#toc1_1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Forward Kinematics Using Homogeneous Transformation Matrix](#toc0_)
In this notebook, we will explain the procedure for implementing "forward kinematics using homogeneous transformation matrices" with Pybullet using a 2-axis robot arm.

(For a manual summarizing the functions available in Pybullet, please refer to [here](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

<br>

In forward kinematics using homogeneous transformation matrices, as shown in the video below:
- Coordinate systems are attached to each link,
- By sequentially transforming coordinates from "Link 1 coordinate system $\Sigma_{1}$" to "Link 2 coordinate system $\Sigma_{2}$" to ... to "End-effector coordinate system $\Sigma_{\mathrm{e}}$",
- The position and orientation of the "End-effector coordinate system $\Sigma_{\mathrm{e}}$" as seen from the "Link 1 coordinate system $\Sigma_{1}$" can be determined.


![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/overview_en.gif)

### <a id='toc1_1_1_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


### <a id='toc1_1_2_'></a>[Initial Setup for pybullet](#toc0_)

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position for GUI mode
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


### <a id='toc1_1_3_'></a>[Generating the Robot Arm](#toc0_)
This time, we will generate a 2-axis robot arm `simple_2d_arm.urdf`.  
The robot is configured as shown in the figure below (the sensors are explained in `robot_arm_sensor.ipynb`).  

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# Load the robot
arm_start_pos = [0, 0, 0.1]  # Set initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True)  # Fix the root link with useFixedBase=True to prevent the robot from falling

# Set the camera position for GUI mode
camera_distance = 1.5
camera_yaw = 180.0  # deg
camera_pitch = -10  # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


### <a id='toc1_1_4_'></a>[Definition of Homogeneous Transformation Matrix in a 2D Plane](#toc0_)
In a 2D plane robot arm, generally:
- The link coordinate system takes the $x$-axis in the direction of the link length.
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/2d_robot_arm_link_cordinate_en.png)

<br>

Therefore, for a robot arm in a 2D plane:
1. Rotate around the origin by $\theta$ (rotation matrix)
2. Translate in the $x$-axis direction (link length direction) by the link length $l$ (translation matrix)

These steps can be used for coordinate transformation.
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/2d_robot_arm_link_homogeneous_matrix_forward_kinematics_en.gif)


The following code defines the "rotation matrix `Hz`" and the "translation matrix `Hp`" in a 2D plane. 

In [4]:
import numpy as np

def Hz(theta):
    """
    Calculate the rotation matrix in a 2D plane (rotation around the z-axis in a 2D plane)

    Parameters
    ----------
    theta : float
        Rotation angle [rad]

    Returns
    -------
    numpy.ndarray
        Rotation matrix
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0], 
                     [np.sin(theta), np.cos(theta), 0], 
                     [0, 0, 1]])

def Hp(x, y):
    """
    Calculate the translation matrix in a 2D plane

    Parameters
    ----------
    x : float
        Translation amount in the x direction
    y : float
        Translation amount in the y direction

    Returns
    -------
    numpy.ndarray
        Translation matrix
    """
    return np.array([[1, 0, x], 
                     [0, 1, y], 
                     [0, 0, 1]])

By combining the "rotation matrix `Hz`" and the "translation matrix `Hp`", you can define the "homogeneous transformation matrix `H`" that performs both the "rotation around the origin" and the "translation" simultaneously.

**【Python Code Example】**
```python
H = Hz(theta) @ Hp(LINK_LENGTH, 0) # Create the homogeneous transformation matrix by combining the rotation matrix and the translation matrix
```

<br>

The homogeneous transformation matrix can be combined using the "chain rule" to form a single transformation matrix. Specifically, the transformation matrices for:

Link 1 coordinate system → Link 2 coordinate system (`H12`)
Link 2 coordinate system → End-effector coordinate system (`H2e`)
can be combined into a single transformation matrix for Link 1 coordinate system → End-effector coordinate system (`H1e`).

**【Python Code Example】**
```python
H12 = Hz(theta1) @ Hp(LINK1_LENGTH, 0) # Homogeneous transformation matrix from Link 1 coordinate system to Link 2 coordinate system
H2e = Hz(theta2) @ Hp(LINK2_LENGTH, 0) # Homogeneous transformation matrix from Link 2 coordinate system to End-effector coordinate system
H1e = H12 @ H2e # Homogeneous transformation matrix from Link 1 coordinate system to End-effector coordinate system
```


![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/direct_transformation_en.gif)


The same principle applies when the number of axes increases. For example:
- The homogeneous transformation matrix from **Link 1 coordinate system** to Link 2 coordinate system (`H12`)
- The homogeneous transformation matrix from Link 2 coordinate system to Link 3 coordinate system (`H23`)
- ...
- The homogeneous transformation matrix from Link n coordinate system to **End-effector coordinate system** (`Hne`)

can be combined into a single homogeneous transformation matrix from **Link 1 coordinate system** to **End-effector coordinate system** (`H1e`).

<br>

### <a id='toc1_1_5_'></a>[Executing Forward Kinematics Using Homogeneous Transformation Matrix](#toc0_)

By running the following code, the "forward kinematics of a 2-axis robot arm using the homogeneous transformation matrix" will be calculated, and the result (the position of the end-effector) will be displayed in pybullet.

* Note: The "position of the end-effector" displayed on the screen is the coordinate position with the "base of the robot arm (= link1 coordinate system) as the origin," not the position with the pybullet world coordinate system as the origin.



In [11]:
LINK1_LENGTH = 0.5   # Length of link1 (length in the z direction of link1 in "simple2d_arm.urdf")
LINK2_LENGTH = 0.55  # Length of link2 (length of link2 + force_sensor_link in "simple2d_arm.urdf")

# Changing these values will change the result ##########
# Define the rotation angles of each link
link1_angle_deg = 60
link2_angle_deg = 40
############################################

link1_angle_rad  = np.deg2rad(link1_angle_deg)
link2_angle_rad  = np.deg2rad(link2_angle_deg)

# Transformation matrix from Link 1 coordinate system to Link 2 coordinate system
H12 = Hz(link1_angle_rad) @ Hp(LINK1_LENGTH, 0) # T12: Link "1" coordinate system -> Link "2" coordinate system
# Transformation matrix from Link 2 coordinate system to End-effector coordinate system
H2e = Hz(link2_angle_rad) @ Hp(LINK2_LENGTH, 0) # T2e: Link "2" coordinate system -> "e"nd effector coordinate system

# Predefine the homogeneous transformation matrix H12e that transforms from Link 1 coordinate system to End-effector coordinate system
H1e = H12 @ H2e # T1e: Link "1" coordinate system -> "e"nd effector coordinate system

x1, y1 = 0, 0 # Origin of Link 1 coordinate system
# Using H12e, find the position of the end effector coordinate system as seen from the origin of the Link 1 coordinate system
oe = H1e @ np.array([x1, y1, 1])
xe, ye = oe[0], oe[1]
print("(xe, ye)=", xe, ye)

# Display the result on the screen
text_position = [0.5, 0.0, 2.0]
life_time = 10.0 # Display duration (seconds)
pybullet.addUserDebugText(f"x, y = ({xe:.2f}, {ye:.2f})", text_position, textSize=2, lifeTime=life_time)

# Actually move the joints to verify that the result of forward kinematics is the same
pybullet.resetJointState(arm_id, 0, link1_angle_rad)
pybullet.resetJointState(arm_id, 1, link2_angle_rad)
pybullet.stepSimulation()

(xe, ye)= 0.15449350228318842 0.9746569660489338


()